Le notebook de preprocessing est stocké sur une branche spécifique (src) du projet précédent, que je récupère via git pour le réutiliser dans mon nouveau pipeline.

In [2]:
# Clonage du projet
!git clone https://github.com/CADETEMMA/Hematology_AI_Project1.git
%cd Hematology_AI_Project1
!git checkout src

# Vérifier que le dossier sourse src existe
!ls




Cloning into 'Hematology_AI_Project1'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 19 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (19/19), 245.76 KiB | 1.60 MiB/s, done.
/content/Hematology_AI_Project1
Branch 'src' set up to track remote branch 'src' from 'origin'.
Switched to a new branch 'src'
preprocessing_py.py  README.md


In [3]:
# Ajout du dossier src au Python path pour pouvoir importer les modules
import sys, os
sys.path.append(os.path.join(os.getcwd(), "src"))

In [3]:
from preprocessing_py import load_data, prepare_train_test


In [7]:
import pandas as pd


In [8]:
url = "https://raw.githubusercontent.com/CADETEMMA/Hematology_AI_Project1/data/aml_tcga_gdc_clinical_data.csv"
df = pd.read_csv(url, sep="\t")


df.head()


,Study ID,Patient ID,Sample ID,Diagnosis Age,Biopsy Site,Cancer Type,Cancer Type Detailed,Last Communication Contact from Initial Pathologic Diagnosis Date,Birth from Initial Pathologic Diagnosis Date,Death from Initial Pathologic Diagnosis Date,...,Project Name,Project State,Race Category,Number of Samples Per Patient,Sample Type,Sample type id,Sex,TMB (nonsynonymous),Patient's Vital Status,Year of Diagnosis
0,aml_tcga_gdc,TCGA-AB-2802,TCGA-AB-2802-03,50,Bone Marrow,Acute Myeloid Leukemia,Acute Myeloid Leukemia,NaN,-18385,365.0,...,Acute Myeloid Leukemia,released,WHITE,1,Primary Blood Derived Cancer - Peripheral Blood,3,Male,NaN,Dead,2001
1,aml_tcga_gdc,TCGA-AB-2803,TCGA-AB-2803-03,61,Bone Marrow,Acute Myeloid Leukemia,Acute Myeloid Leukemia,NaN,-22584,792.0,...,Acute Myeloid Leukemia,released,WHITE,1,Primary Blood Derived Cancer - Peripheral Blood,3,Female,NaN,Dead,2001
2,aml_tcga_gdc,TCGA-AB-2804,TCGA-AB-2804-03,30,Bone Marrow,Acute Myeloid Leukemia,Acute Myeloid Leukemia,2557.0,-11203,NaN,...,Acute Myeloid Leukemia,released,WHITE,1,Primary Blood Derived Cancer - Peripheral Blood,3,Male,NaN,Alive,2001
3,aml_tcga_gdc,TCGA-AB-2805,TCGA-AB-2805-03,76,Bone Marrow,Acute Myeloid Leukemia,Acute Myeloid Leukemia,NaN,-28124,577.0,...,Acute Myeloid Leukemia,released,WHITE,1,Primary Blood Derived Cancer - Peripheral Blood,3,Male,NaN,Dead,2002
4,aml_tcga_gdc,TCGA-AB-2806,TCGA-AB-2806-03,46,Bone Marrow,Acute Myeloid Leukemia,Acute Myeloid Leukemia,NaN,-16892,945.0,...,Acute Myeloid Leukemia,released,WHITE,1,Primary Blood Derived Cancer - Peripheral Blood,3,Male,NaN,Dead,2002


In [9]:
print(df.columns.tolist())


['Study ID', 'Patient ID', 'Sample ID', 'Diagnosis Age', 'Biopsy Site', 'Cancer Type', 'Cancer Type Detailed', 'Last Communication Contact from Initial Pathologic Diagnosis Date', 'Birth from Initial Pathologic Diagnosis Date', 'Death from Initial Pathologic Diagnosis Date', 'Ethnicity Category', 'Fraction Genome Altered', 'Is FFPE', 'Mutation Count', 'Oncotree Code', 'Overall Survival (Months)', 'Overall Survival Status', 'Other Patient ID', 'Other Sample ID', 'Patient Primary Tumor Site', 'Prior Malignancy', 'Prior Treatment', 'Project Identifier', 'Project Name', 'Project State', 'Race Category', 'Number of Samples Per Patient', 'Sample Type', 'Sample type id', 'Sex', 'TMB (nonsynonymous)', "Patient's Vital Status", 'Year of Diagnosis']


In [10]:
# Créer la variable cible
df["target"] = df["Overall Survival Status"].map({
    "1:DECEASED": 1,
    "0:LIVING": 0
})

df["target"].value_counts()


,count
target,
1,133
0,67


La varribale cible est déséquilibrée.

In [14]:
# définir les features
num_features = [
    "Diagnosis Age",
    "Mutation Count",
    "Fraction Genome Altered"
]

cat_features = [
    "Sex",
    "Prior Malignancy",
    "Prior Treatment"
]


In [15]:
# preprocessing + train/test split (sans data leakage)
X_train, X_test, y_train, y_test, preprocessor = prepare_train_test(
    df=df,
    target_col="target",
    num_features=num_features,
    cat_features=cat_features,
    test_size=0.2,
    random_state=42
)

print(X_train.shape, X_test.shape)


(160, 9) (40, 9)


In [16]:
# import des modèles ML
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix


In [17]:
# régression logistique
logreg = LogisticRegression(max_iter=1000, random_state=42)
logreg.fit(X_train, y_train)

y_pred_log = logreg.predict(X_test)
y_proba_log = logreg.predict_proba(X_test)[:, 1]


In [18]:
# random forest
rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)
y_proba_rf = rf.predict_proba(X_test)[:, 1]


In [19]:
# XGBoost
xgb = XGBClassifier(
    eval_metric="logloss",
    random_state=42
)

xgb.fit(X_train, y_train)

y_pred_xgb = xgb.predict(X_test)
y_proba_xgb = xgb.predict_proba(X_test)[:, 1]


In [20]:
# évaluation clinique des modèles
def evaluate_model(name, y_true, y_pred, y_proba):
    acc = accuracy_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_proba)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)

    print(f"--- {name} ---")
    print(f"Accuracy     : {acc:.3f}")
    print(f"AUC ROC      : {auc:.3f}")
    print(f"Sensitivity  : {sensitivity:.3f}")
    print(f"Specificity  : {specificity:.3f}\n")

evaluate_model("Logistic Regression", y_test, y_pred_log, y_proba_log)
evaluate_model("Random Forest", y_test, y_pred_rf, y_proba_rf)
evaluate_model("XGBoost", y_test, y_pred_xgb, y_proba_xgb)


--- Logistic Regression ---
Accuracy     : 0.650
AUC ROC      : 0.698
Sensitivity  : 0.889
Specificity  : 0.154

--- Random Forest ---
Accuracy     : 0.725
AUC ROC      : 0.613
Sensitivity  : 0.852
Specificity  : 0.462

--- XGBoost ---
Accuracy     : 0.725
AUC ROC      : 0.588
Sensitivity  : 0.815
Specificity  : 0.538



Régression logistique :
Sensibilité à 89% -> très bonne
AUC -> la plus élevée des 3 modèles
spécificité -> très faible 15% (bcp de faux-positifs)

Le modèle dépend de l'objectif clinique.
XGBoost -> surdimensionné, pas adapté à la taille, bruit du dataset

La régression logistique présente la meilleure capacité discriminante (AUC) et la meilleure sensibilité, critère prioritaire en hématologie afin de limiter les faux négatifs.
Malgré une spécificité plus faible, ce modèle est retenu comme baseline clinique interprétable.
Les modèles plus complexes n'apportent pas de gain significatif de performance sur ce dataset de taille limitée.


In [21]:
# variables finales utilisées par le modèle
feature_names = preprocessor.get_feature_names_out()

print("Features utilisées :")
for f in feature_names:
    print("-", f)

print("\nNombre total de features :", len(feature_names))


Features utilisées :
- num__Diagnosis Age
- num__Mutation Count
- num__Fraction Genome Altered
- cat__Sex_Female
- cat__Sex_Male
- cat__Prior Malignancy_False
- cat__Prior Malignancy_True
- cat__Prior Treatment_False
- cat__Prior Treatment_True

Nombre total de features : 9
